In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import openai
import json
import ijson
import os
import requests
from retrying import retry
from pywikidata import Entity
openai.api_key = os.getenv('KEY')

openai.base_url = "https://api.vsegpt.ru:6070/v1/"

/Users/aa/vmk/diplom/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 756
    true_answers = 483
    answers = []
    for j, item in enumerate(array_items):
        if j < 756:
            continue
        messages = []
        i += 1

        question = item["question_eng"]
        try:
            answer = ""
            for x in item["answers"][0]["wd_names"]["en"]:
                answer += x.lower() + ' '
        except:
            continue
        prompt = "Answer shortly in 1-3 words. If answer is a number, print digits"
        # prompt = "Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже, при этом не забывай о пунктуации."
        # messages.append({"role": "user", "content": prompt})
        messages.append({"role": "user", "content": prompt + ' ' + question})
        response = ""
        try:
            response_big = openai.chat.completions.create(
                model="openai/gpt-3.5-turbo-1106",
                messages=messages,
                temperature=0.1,
            )
        except:
            continue
        response = response_big.choices[0].message.content.lower()
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer
        })
        if response in answer:
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers, indent=4)
        with open("questions_answers_rubq.json", "w") as json_file:
            json_file.write(json_data)

757 484
758 484
759 485
760 485
761 486
762 486
763 487
764 488
765 489
766 489
767 489
768 490
769 490
770 491
771 491
772 491
773 492
774 492
775 493
776 493
777 494
778 495
779 495
780 496
781 497
782 498
783 499
784 500
785 501
786 501
787 501
788 502
789 503
790 504
791 505
792 506
793 507
794 508
795 508
796 508
797 509
798 509
799 510
800 511
801 511
802 512
803 512
804 513
805 513
806 514
807 514
808 515
809 516
810 516
811 517
812 517
813 517
814 517
815 517
816 517
817 517
818 518
819 518
820 518
821 518
822 519
823 520
824 520
825 521
826 522
827 522
828 522
829 523
830 523
831 523
832 523
833 523
834 524
835 525
836 525
837 525
838 525
839 525
840 525
841 525
842 525
843 525
844 526
845 527
846 528
847 528
848 528
849 529
850 529
851 529
852 530
853 530
854 530
855 531
856 531
857 531
858 532
859 533
860 533
861 534
862 535
863 535
864 536
865 537
866 538
867 538
868 538
869 539
870 539
871 539
872 539
873 539
874 540
875 541
876 542
877 542
878 542
879 542
880 543
881 543


In [3]:
df_test = pd.read_csv('archive/n_annotated_wd_data_test_answerable.csv')
df_test.head()

df_train = pd.read_csv('archive/n_annotated_wd_data_train_answerable.csv')

df_valid = pd.read_csv('archive/n_annotated_wd_data_valid_answerable.csv')


In [4]:
df_test['q'][1223]

'Name a person born in queens\n'

In [10]:
# @retry(stop_max_attempt_number=3, wait_fixed=20000)  # Retry 3 times with a fixed delay of 2 seconds
def send_request(question_text):
    url = 'http://5.35.10.151:8080/pipeline/m3m'  # Replace with your server's IP address and port
    headers = {'Content-Type': 'application/json'}
    data = {'text': question_text}

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200 or response.status_code == 307:
        return response.json()  # Assuming the response is in JSON format
    else:
        print(f"Error: {response.status_code}")
        raise Exception

In [11]:
send_request("Where?")

{'answers': ['Q65',
  'Q60',
  'Q30',
  'Q8652',
  'Q1408',
  'Q5148553',
  'Q79202',
  'Q844652',
  'Q497155',
  'Q99',
  'Q1649',
  'Q434841',
  'Q1626895',
  'Q84',
  'Q3503936',
  'Q622668',
  'Q13974',
  'Q621364',
  'Q16242963',
  'Q1163074',
  'Q3141',
  'Q1323594',
  'Q145',
  'Q744357',
  'Q40',
  'Q362',
  'Q860626',
  'Q168383',
  'Q223193',
  'Q314771',
  'Q310060',
  'Q316977'],
 'scores': [0.00830150581896305,
  0.007837696000933647,
  0.007544948253780603,
  0.007424716372042894,
  0.007360268849879503,
  0.007226359099149704,
  0.007179952226579189,
  0.0071798283606767654,
  0.007167454808950424,
  0.007130903657525778,
  0.007121264934539795,
  0.007106325589120388,
  0.007065795361995697,
  0.007046486716717482,
  0.007020450197160244,
  0.006990097463130951,
  0.006987355649471283,
  0.006981966085731983,
  0.006981737911701202,
  0.006953729782253504,
  0.006938112899661064,
  0.006937965750694275,
  0.006920847110450268,
  0.006917767226696014,
  0.006908034905791

In [12]:
answers = []
i = 0
true_answers = 0
for question, answer in zip(df_test['q'], df_test['e2']):
    messages = []
    i += 1

    response = ""
    try:
        response_data = send_request(question)
        response = Entity(response_data['answers'][0]).label.lower()
    except Exception as e:
        response = str(e)
    answers.append({
        "question": question,
        "answer": response,
        "correct_answer": answer.lower(),
        "is_correct": answer.lower() in response
    })
    if answer.lower() in response:
        true_answers += 1
    print(i, true_answers)
    json_data = json.dumps(answers, indent=4)
    with open("questions_answers.json", "w") as json_file:
        json_file.write(json_data)

1 0
2 1
3 1
4 1
5 2
6 2
7 2
8 2
9 2
10 3
11 3
12 3
13 3
14 3
15 3
16 4
17 4
18 5
19 5
20 5
21 5
Error: 500
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 6
30 7
31 8
32 8
33 8
34 8
35 8
36 8
37 9
38 9
39 9
40 10
41 10
Error: 500
42 10
43 10
Error: 500
44 10
45 10
46 11
47 11
48 11
49 11
50 11
Error: 500
51 11
52 11
53 11
54 12
55 13
56 13
57 13
58 13
59 13
60 13
61 13
62 13
63 13
64 14
65 14
66 15
Error: 500
67 15
68 15
69 16
70 17
71 18
72 18
73 18
74 18
75 18
76 19
77 19
78 19
79 19
Error: 500
80 19
81 19
82 19
83 19
84 20
85 20
86 21
87 21
88 22
89 22
Error: 500
90 22
91 22
92 22
Error: 500
93 22
94 22
95 22
96 22
97 22
98 22
99 23
100 23
101 23
102 23
103 23
104 23
105 23
106 23
107 23
108 24
109 24
110 25
111 25


KeyboardInterrupt: 

In [ ]:

for question, answer in zip(df_valid['q'][:2], df_valid['e2']):
    record = convert_conversation(question, answer, system_message)
    validation.append(record)
    

5621